# 1 问题引入

聚类分析(cluster analysis)，也被称为数据分割(data segmentation)。是指将一组对象的集合分组或分割为几个子集或类(cluster)，每个类别中的对象之间的相似度要高于与其他类别中对象的相似度，可以用不同的量度来描述相似度。有时候，类别之间也具有层级关系(hierarchy)。聚类分析的核心在于如何度量对象之间的相似度或非相似度(dissimilarity)，度量的方式类似于监督学习中的损失函数。下面将先讨论一般性的聚类问题，然后针对文本聚类，实现两种常用的聚类方法。

# 2 计算模型

## 2.1 相似度矩阵(proximity matrix)

相似度是指两个对象之间的相似程度，可以用一个$N\times N$的矩阵$\mathbf{D}$来表示，其中$N$代表对象的个数，$d_{ii'}$记录了第$i$个对象和第$i'$个对象的相似度。相似度矩阵一般作为聚类算法的输入。

大多数聚类算法都假定相似度矩阵的所有元素都是非负的，对角线元素均为0。大多数算法也假定相似度矩阵是对称的，如果不对称，则用$(\mathbf{D}+\mathbf{D^T})/2$来代替$D$。在严格意义上，大多数情况下的相似度并不具有距离(distance)的含义，因为三角不等式在这种情况下不会成立($d_{ii'}\leq d_{ik}+d_{i'k}$)。

我们用$x_{ij}$来表示对象的属性(attribute)，$i=1,2,\cdots,N$，$j=1,2,\cdots,p$。定义非相似度$d_j(x_{ij},x_{i'j})$，表示第$j$组属性的非相似度，则：
$$
D(x_i,x_{i'})=\sum_{j=1}^p d_j(x_{ij},x_{i'j})
$$
就是对象$i$和$i'$之间的非相似度。最常用的就是对象之间的平方距离：
$$
d_j(x_{ij},x_{i'j})=(x_{ij}-x_{i'j})^2
$$
对于一些不能量化的属性，平方距离将不再适用，可以用一些其他的方法表示两个对象之间的非相似度。

## 2.2 聚类算法

目前主要有以下几种分类算法：组合算法，混合模型和模式匹配。

组合算法直接观察数据，不考虑数据内部服从的概率模型。混合模型假设数据都服从某个独立同分布，整体数据的分布是某几个不同参数分布的叠加，每个分布就是一个类别，通常通过最大似然估计来确定各个模型的参数。模式匹配就是直接估计出概率分布函数的模式，最符合某个独立模式的一批样本属于同一个分类。

下面主要讨论组合算法中的K-means方法和混合模型中的高斯混合模型方法。

## 2.3 K-means聚类算法

最受欢迎的聚类算法就是直接学习出每个样本所属的分类而不去考虑数据的概率模型。每个样本都用一个整数$i\in\{1,\cdots,N\}$来标记，人为的确定某个整数$K<N$，每个样本所属的类别为$k\in\{1,\cdots,K\}$。每个样本将被分为某个唯一的类别。这些从样本编号到类别编号的映射关系就是分类器$k=C(i)$。我们需要基于每对样本值的非相似度，找到某个最优的分类器$k=C^*(i)$。最优的目标就是调整映射关系，最小化损失函数(表征距实现聚类目标远近的程度)。为了将相似的样本都划分到同一类别，定义如下损失函数：
$$
W(C)=\frac{1}{2}\sum_{k=1}^K\sum_{C(i)=k}\sum_{C(i')=k}d(x_i,x_{i'})
$$
即相同类别的样本之间的相似度往往很高。

K-means算法是比较常用的迭代聚类方法，它适用于样本属性均为可量化类型的情况。K-means选择平方欧氏距离来衡量样本间的非相似度：
$$
d(x_i,x_{i'})=\sum_{j=1}^p(x_{ij}-x_{i'j})=||x_i-x_{i'}||^2
$$
则上述损失函数就可以简化为：

\begin{align*}
 W(C)&= \frac{1}{2}\sum_{k=1}^K\sum_{C(i)=k}\sum_{C(i')=k}d(x_i,x_{i'}) \\
     &= \sum_{k=1}^K N_k\sum_{C(i)=k} ||x_i-\overline{x}_k|| ^2
\end{align*} 

其中，$\overline{x}_k=(\overline{x}_{1k},\cdots,\overline{x}_{pk},)$表示第$k$类样本向量的均值，$N_k=\sum_{i=1}^N I(C(i)=k)$。因此，我们优化的目标就是找到某个最优的分类器：
$$
C^*=\min_C \sum_{k=1}^K N_k\sum_{C(i)=k}||x_i-\overline{x}_k||^2
$$
注意到，对于任意样本集合都有：
$$
\overline{x}_S=argmin_m\sum_{i\in S}||x_i-m||^2
$$
即求解如下的优化问题：
$$
\min_{C,\{m_k\}_1^K} \sum_{k=1}^K N_k\sum_{C(i)=k}||x_i-m_k||^2
$$

下面是K-means聚类的算法流程：
1. 生成初始值$m_k$
2. 对于每个$i$，更新分类器的值：$C(i)=argmax_{1\leq k\leq K}||x_i-m_k||^2$
3. 更新均值向量的值：$m_k=\frac{1}{N_k}\sum_{C(i)=k}x_i$
4. 如果未收敛，则回到步骤2，直至收敛。

## 2.4 高斯混合模型

高斯混合模型假设每个类别的数据都服从相互独立的高斯分布，通过极大似然估计和EM算法确定每个高斯分布的参数值，最后根据确定的高斯分布求出样本属于某高斯模型的概率，取得最大概率的高斯分布所属的类别就是样本的类别。

高斯模型的线性混合：
$$
p(x)=\sum_{k=1}^K \pi_k \mathcal{N}(\mathbf{x}|\mathbf{\mu}_k, \mathbf{\Sigma}_k)
$$
混合系数为：
$$
\sum_{k=1}^K \pi_k=1,0\leq\pi_k\leq 1
$$
上述公式其实可以看做一种先验概率：
$$
p(x)=\sum_{k=1}^K p(k)p(\mathbf{x}|k)
$$
下面估计相应类别标签$k$的后验概率(responsibilities)：
\begin{align*}
\gamma_k=p(k|\mathbf{x})=& \frac{p(k)p(\mathbf{x}|k)}{p(\mathbf{x})} \\
                        =& \frac{\pi_k\mathcal{N}(\mathbf{x}|\mathbf{\mu}_k, \mathbf{\Sigma}_k)}{\sum_{j=1}^K\pi_j\mathcal{N}(\mathbf{x}|\mathbf{\mu}_j, \mathbf{\Sigma}_j)}
\end{align*}

使用极大似然估计：
$$
\ln p(D|\mathbf{\pi},\mathbf{\mu}, \mathbf{\Sigma})=\sum_{n=1}^N \ln\{\sum_{k=1}^K \pi_k\mathcal{N}(\mathbf{x}_n|\mathbf{\mu}_k, \mathbf{\Sigma}_k)\}
$$

由于极大似然估计的解不是封闭形式(变量之间互为耦合)，故采用EM算法进行迭代求解。

首先给参数设定一个初始值，通过以下两个步骤更新参数直至收敛：

E-step: 估计后验概率
$$
\gamma_k(x_i)= \frac{\pi_k\mathcal{N}(\mathbf{x}_i|\mathbf{\mu}_k, \mathbf{\Sigma}_k)}{\sum_{j=1}^K\pi_j\mathcal{N}(\mathbf{x_i}|\mathbf{\mu}_j, \mathbf{\Sigma}_j)}
$$
M-step：采用MLE更新参数
$$
\mu_k=\frac{1}{N_k}\sum_{i=1}^N \gamma_k(x_i)
$$
$$
\Sigma_k=\frac{1}{N_k}\sum_{i=1}^N \gamma_k(x_i)(x_i-\mu_k)(x_i-\mu_k)^T
$$
$$
\pi_k=\frac{1}{N}\sum_{i=1}^N \gamma_k(x_i)
$$

分类时，只要把每个数据点带入到每个混合成分$C_k$中，当概率大于一定阈值时便认为其属于$C_k$类。

# 3 编程实现

## 3.1 K-means

In [1]:
import scipy.io
import scipy
import random
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import importlib
import sys 
import os

In [18]:
# load data
load_data = scipy.io.loadmat('data/news_data.mat')
news_data = load_data['data']
news_labels = load_data['labels']

# shuffle
zipped_data = list(zip(news_data, news_labels))  
random.seed(0)
random.shuffle(zipped_data)
new_zipped_data = list(map(list, zip(*zipped_data)))  
news_data, news_labels = np.array(new_zipped_data[0]), np.array(new_zipped_data[1])  

# split data into training and test sets
training_data = news_data[:1000, 4900:]
training_labels = news_labels[:1000]
test_data = news_data[15000:, :]
test_labels = news_labels[15000:]

In [20]:
# K-means
class KMeans:
    def __init__(self):
        # K
        self.K = 0
        # assignments
        self.C = 0
        # mean vectors
        self.m = 0
        # loss function
        self.loss_fn = 0
        # N data points
        self.N = 0
        # dimension
        self.d = 0
        
    def generate_random_means(self, data):
        m = np.zeros((self.K, self.d))
        for k in range(self.K):
            m[k] = data[int(random.random()*self.N)]
        return m
    
    def squared_euclidean_dist(self, u, v):
        diff = u - v
        return sum(diff*diff)
    
    def fit(self, data, K, max_iter):
        print("Start fitting...")
        self.K = K
        self.N = data.shape[0]
        self.d = data.shape[1]
        self.C = np.zeros((self.N, 1))
        self.loss_fn = np.zeros((self.N, 1))
        
        cnt = 0
        self.m = self.generate_random_means(data)
        
        while cnt < max_iter:
            changed = False
            
            for i in range(self.N):
                min_dissimilarity = float('inf')
                min_k = 0
                for k in range(self.m.shape[0]):
                    dissimilarity = self.squared_euclidean_dist(data[i], self.m[k])
                    if dissimilarity < min_dissimilarity:
                        min_dissimilarity = dissimilarity
                        min_k = k
                self.loss_fn[i] = min_dissimilarity
                if self.C[i] != min_k:
                    self.C[i] =min_k
                    changed = True
                    
            for k in range(self.m.shape[0]):
                data_k = data[self.C.ravel()==k]
                if data_k.shape[0] != 0:
                    self.m[k] = np.sum(data_k, axis=0) / data_k.shape[0]
                
            cnt += 1
            if not changed:
                print("converged!")
                break
        
        print("Finish fitting !")

In [21]:
km = KMeans()
km.fit(training_data, 20, 100)

Start fitting...
converged!
Finish fitting !


In [22]:
km.C[:20]

array([[ 7.],
       [15.],
       [16.],
       [10.],
       [ 6.],
       [ 3.],
       [16.],
       [ 7.],
       [ 7.],
       [16.],
       [16.],
       [19.],
       [16.],
       [16.],
       [10.],
       [ 7.],
       [ 6.],
       [ 7.],
       [16.],
       [ 3.]])

## 3.2 Gaussian Mixture Model

In [2]:
# load data
load_data = scipy.io.loadmat('data/news_data.mat')
news_data = load_data['data']
news_labels = load_data['labels']

# shuffle
zipped_data = list(zip(news_data, news_labels))  
random.seed(0)
random.shuffle(zipped_data)
new_zipped_data = list(map(list, zip(*zipped_data)))  
news_data, news_labels = np.array(new_zipped_data[0]), np.array(new_zipped_data[1])  

# split data into training and test sets
training_data = news_data[:100, 4990:]
training_labels = news_labels[:100]
test_data = news_data[15000:, :]
test_labels = news_labels[15000:]

In [26]:
# K-means
class GMM:
    def __init__(self):
        # K
        self.K = 0
        # N data points
        self.N = 0
        # dimension
        self.d = 0
        # prior
        self.pi = 0
        # means
        self.u = 0
        # covarience
        self.sigma = 0
        # responsibility
        # self.r = 0
        
    def gaussian_pdf(self, x, u, sigma):
        if np.linalg.det(sigma)==0:
            return 0
        if -(x-u).dot(scipy.linalg.pinv(sigma).dot(x.T-u.T))/2>100:
            return 0
        return 1/(np.sqrt(abs(np.linalg.det(sigma))))*np.exp(-(x-u).dot(scipy.linalg.pinv(sigma).dot(x.T-u.T))/2)
        
    # E step
    def expectation_step(self, x, k):
        numerator = self.pi[k]*self.gaussian_pdf(x, self.u[k], self.sigma[k])
        denominator = sum([self.pi[j]*self.gaussian_pdf(x, self.u[j], self.sigma[j]) for j in range(self.K)])
        if denominator==0:
             return 0
        return numerator/denominator
        
    # M step
    def maximization_step(self, data, k):
        N_k = self.r.sum()
        pi_k = N_k/self.N
        u_k = np.zeros(self.d)
        for i in range(self.N):
            u_k += self.r[k][i]*data[i]
        sigma_k = np.zeros((self.d,self.d))
        for i in range(self.N):
            sigma_k += self.r[k][i]*(data[i].T-u_k.T).dot(data[i]-u_k)
        sigma_k = sigma_k/N_k
        return pi_k, u_k, sigma_k
        
    
    def fit(self, data, K, max_iter):
        print("Start fitting...")
        self.K = K
        self.N = data.shape[0]
        self.d = data.shape[1]
        self.pi = np.ones((self.K, 1))/self.K
        self.u = np.random.rand(self.K, self.d)
        self.sigma = np.zeros((self.K, self.d, self.d))
        for k in range(self.K):
            self.sigma[k] = np.random.rand(self.d, self.d)
        self.r = np.zeros((self.K, self.N))
        
        cnt = 0        
        while cnt < max_iter:
            changed = False
            
            # E step
            for k in range(self.K):
                for i in range(self.N):
                    r_ki = self.expectation_step(data[i], k)
                    if r_ki != self.r[k][i]:
                        self.r[k][i] = r_ki
                        changed = True
            
            # M step
            for k in range(self.K):
                pi_k, u_k, sigma_k = self.maximization_step(data, k)
                if pi_k.all()!=self.pi[k].all() or u_k.all()!=self.u[k].all() or sigma_k.all()!=self.sigma[k].all():
                    self.pi[k], self.u[k], self.sigma[k] = pi_k, u_k, sigma_k
                    changed = True
                
            cnt += 1
            if not changed:
                print("converged!")
                break
        print("Finish fitting !")
        
    def predict(self, x):
        k_pred = 0
        max_prob = 0
        for k in range(self.K):
            prob = 1/(np.sqrt(abs(np.linalg.det(self.sigma[k]))))*np.exp(-(x-self.u[k]).dot(scipy.linalg.pinv(self.sigma[k]).dot(x.T-self.u[k].T))/2)
            if prob > max_prob:
                max_prob = prob
                k_pred = k
        return k_pred

In [27]:
gmm = GMM()
gmm.fit(training_data, 20, 5)

Start fitting...
converged!
Finish fitting !


In [28]:
gmm.predict(training_data[0])

12

In [29]:
gmm.predict(training_data[1])

6

In [30]:
gmm.predict(training_data[2])

17

# 4 模型评估